# PanQEC basics

In this tutorial, we will learn how the library is structure and what we can do with it.

## Codes, noise models and decoders

PanQEC is made of three main ingredients: the codes, the noise models and the decoders. Let's take a look at an example of how those components can interact.

In [1]:
from panqec.codes import Toric2DCode
from panqec.error_models import PauliErrorModel
from panqec.decoders import Toric2DPymatchingDecoder

%load_ext autoreload
%autoreload 2

Let's start by instantiating the following elements:
1. A 2D toric code of lattice size $L=4$
2. A Pauli error model where 20% of errors are Pauli $X$ errors, 30% are Pauli $Y$, and 50% Pauli $Z$, and the total probability of error is $p=0.1$
3. A minimum-weight perfect matching decoder based on the library [PyMatching](pymatching.readthedocs.io)

In [2]:
code = Toric2DCode(4)

error_model = PauliErrorModel(0.2, 0.3, 0.5)
p = 0.1

decoder = Toric2DPymatchingDecoder(error_model, p)

We can then use the error model to generate some errors, from which we can extract a syndrome

In [14]:
errors = error_model.generate(code, p)
syndrome = code.measure_syndrome(errors)

print("Errors: ", errors)
print("Syndrome: ", syndrome)

Errors:  [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0
 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 1]
Syndrome:  [1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 0 0 0 1]


And finally we can decode the syndrome using our matching decoder, and check the decoding succeeded

In [18]:
# Obtain the correction by decoding the syndrome
correction = decoder.decode(code, syndrome)
print("Correction: ", correction)

# Get the remaining error once the correction has been applied
residual_error = (correction + errors) % 2
print("Residual error: ", residual_error)

# Check whether the residual error is in the codespace 
# (i.e. whether there is any remaining excitation)
in_codespace = code.in_codespace(residual_error)
print("Is in codespace: ", in_codespace)

# Get the logical errors
logical_errors = code.logical_errors(residual_error)
print("Logical errors: ", logical_errors)

# Check whether the decoding succeeded
success = not code.is_logical_error(residual_error) and in_codespace
print("Success: ", success)

Correction:  [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1]
Residual error:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0
 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0]
Is in codespace:  True
Logical errors:  [0 0 1 1]
Success:  False
